# Topic Extraction in Newsfeeds
* Autor: Prof. Dr. Johannes Maucher
* Datum: 17.11.2015

[Übersicht Ipython Notebooks im Data Mining Praktikum](Data Mining Praktikum.html)

#Einführung
## Lernziele:

In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* __RSS Feeds:__ Struktur von RSS Feeds analysieren und parsen mit dem _Universal Feed Parser_. 
* __Dokument Analyse:__ Die Häufigkeit aller Worte in einem Dokument (Inhalt des RSS Feeds) zählen und in einem Array verwalten. 
* __Merkmalsextraktion:__ Bestimmung von Merkmalen (hier auch: __Topics__) (Allgemein spricht man von Merkmalen. Im Fall, dass die NNMF auf Dokumente angewandt wird, werden die Merkmale auch mit __Topics__ oder __Themen__ bezeichnet) mit der \emph{Non Negative Matrix Factorization}.
* __Zuordnung__: Wie setzen sich die Topics aus den Wörtern zusammen? Wie stark sind die gefundenen Topics in den Artikeln vertreten?
* __Dokument Clustering:__ Mit der NNMF kann auch ein Clustering realisiert werden. Jeder Topic repräsentiert ein Cluster. Jedes Dokument wird dem Cluster zugeordnet, dessen Topic am stärksten in ihm vertreten ist. 

Sämtliche Verfahren und Algorithmen werden in Python implementiert.

## Theorie zur Vorbereitung

Stellen Sie sich vor Sie möchten in eine eigene Webseite die RSS Feeds einer Menge von Nachrichtenservern einbinden. Da die unterschiedlichen Server wahrscheinlich Artikel zu den gleichen Themen anbieten, werden die Inhalte einiger Artikel ähnlich sein. Mit der __Nicht Negativen Matrixfaktorisierung (NNMF)__ kann für eine große Menge von Dokumenten eine Menge von Themen (Topics) ermittelt werden, auf die sich die Dokumente beziehen. Damit ist es u.a. möglich
* die Dokumente thematisch zu ordnen
* zu jedem Thema nur ein Dokument anzuzeigen

### Ähnlichkeiten bestimmen und relevante Merkmale extrahieren

Eine Sammlung von Dokumenten - in diesem Versuch die Menge aller Nachrichten der angegebenen Feeds - kann in einer Artikel/Wort-Matrix repräsentiert werden. Jede Zeile dieser Matrix gehört zu einem Dokument. Für jedes Wort, das mindestens in einem der Dokumente vorkommt, ist eine Spalte vorgesehen. Das Matrixelement in Zeile $i$, Spalte $j$ beschreibt wie häufig das Wort in Spalte $j$ im zur Zeile $i$ gehörenden Dokument vorkommt.

Unter der Annahme, dass Artikel umso ähnlicher sind, je mehr Worte in diesen gemeinsam vorkommen, kann auf der Grundlage dieser Matrix die Ähnlichkeit zwischen den Artikeln berechnet werden. Hierzu könnte die Matrix z.B. einfach einem _Hierarchischen Clustering_ übergeben werden. Das hierarchische Clustering weist jedoch im Fall einer großen Menge von zu vergleichenden Objekten zwei wesentliche Nachteile auf: Erstens ist die wiederholte Berechnung der Distanzen zwischen allen Artikeln/Clustern extrem rechenaufwendig, zweitens ist die Darstellung einer großen Anzahl von Objekten im Dendrogramm nicht mehr übersichtlich. 

Für das Auffinden von Assoziationen zwischen Dokumenten hat sich in den letzten Jahren die Methode der __Nicht-Negativen Matrix Faktorisierung (NNMF)__ etabliert. Mit dieser Methode kann eine Menge von wesentlichen Merkmalen berechnet werden, anhand derer sich die Dokumente clustern lassen, d.h. Dokumente des gleichen Clusters repräsentieren das gleiche Merkmal (Thema). Ein solches Merkmal wird durch eine Menge von Worten beschrieben, z.B. $\{$ _Paris, terror, IS_ $\}$  oder $\{$_refugee, syria, border_ $\}$. Neben der Merkmalsextraktion stellt die relativ geringe Komplexität einen weiteren Vorteil der NNMF dar. Durch die Darstellung der Artikel/Wort-Matrix als Produkt von 2 Faktormatrizen müssen deutlich weniger Einträge gespeichert werden.

### Nicht Negative Matrixfaktorisierung: Die Idee

Die Artikel/Wort-Matrix wird im Folgenden mit $A$ bezeichnet. Sie besitzt $r$ Zeilen und $c$ Spalten, wobei $r$ die Anzahl der Artikel und $c$ die Anzahl der relevanten Worte in der Menge aller Artikel ist. Durch Multiplikation der Matrix $A$ mit dem Vektor $v$ (_wordvec_: Vektor der alle relevanten Worte enthält) werden die Worte den Artikeln $a$ (_articletitles_: Vektor der alle Artikeltitel enthält) zugeordnet:

$$
a=A*v.
$$

Die Idee der NNMF besteht darin die Matrix $A$ als Produkt zweier Matrizen $W$ und $H$ darzustellen,

$$
A=W*H
$$

wobei alle Elemente in $W$ und $H$ größer oder gleich Null sein müssen. Die Matrixmultiplikation erfordert, dass die Anzahl der Zeilen $m$ in $H$ gleich der Anzahl der Spalten in $W$ sein muss. 
Durch die Faktorisierung der Matrix $A$ wird die Zuordnung der Wörter des Wortvektors $v$  zu den Artikeln des Vektors $a$ in zwei Stufen zerlegt. 

$$
f = H*v
$$
$$
a = W*f 
$$

In der ersten Stufe werden durch die Multiplikation von $v$ mit der Matrix $H$ die Wörter einem sogenannten Merkmalsvektor $f$ mit $m$ Elementen zugewiesen. In der zweiten Stufe werden durch die Multiplikation des Merkmalsvektor $f$ mit der Matrix $W$ die einzelnen Merkmale den Artikeln in $a$ zugeordnet. Die Matrix $H$ definiert also aus welchen Wörtern die Merkmale gebildet werden. Sie wird deshalb __Merkmalsmatrix__ genannt. Die Matrix $W$ hingegen beschreibt mit welchem Gewicht die einzelnen Merkmale in den verschiedenen Artikeln auftreten. Sie wird deshalb __Gewichtungsmatrix__ genannt.

Daraus folgt: Wenn eine Faktorisierung der Matrix $A$ gefunden wird, dann werden damit auch relevante Merkmale, also die Themen, definiert, hinsichtlich derer die Artikel effizient kategorisiert werden. Durch die Matrixfaktorisierung wird eine __Merkmalsextraktion__ realisiert. 

### Berechnung der Matrixfaktoren

Für die Berechnung der Faktoren wurde in [Lee, Algortihms for Non-negative Matrix Factorisation](http://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf) eine iterative Methode vorgestellt, die derzeit wohl am häufigsten angewandt wird und auch in dieser Übung implementiert werden soll. Der Algorithmus besteht aus folgenden Schritten:
* Gebe die zu faktorisierende Matrix $A$ ein. $r$ sei die Anzahl der Zeilen und $c$ die Anzahl der Spalten von $A$.
* Wähle die Anzahl $m$ der Merkmale, mit $m<c$. _Tipp:_ Für $m$ sollte zunächst ein Wert im Bereich $15$ bis $30$ gewählt werden.
* Lege eine $m \times c$ Matrix $H$ an mit initial zufälligen Elementen (Anwendung der numpy Funktion _random.random()_)
* Lege eine $r \times m$ Matrix $W$ an mit initial zufälligen Elementen (Anwendung der numpy Funktion _random.random()_)
* Wiederhole bis maximale Anzahl der Iteration erreicht oder Kosten $k$ unter vordefinierter Schwelle:

	* Berechne aktuelles Produkt $B=W*H$ und bereche die Kostenfunktion 
		$$
			k=\left\| A - B \right\|^2 = \sum\limits_{i,j} \left(A_{i,j} - B_{i,j}\right)^2
		$$ 
	* Anpassung der Matrix $H$ durch folgende Neuberechnung der Matrixelemente
    
		$$
		H_{i,j} := H_{i,j} \frac{(W^T*A)_{i,j}}{(W^T*W*H)_{i,j}}
		$$
        
	* __Nach__ der Anpassung der Matrix $H$: Anpassung der Matrix $W$ durch folgende Neuberechnung der Matrixelemente
    
		$$
		W_{l,i} := W_{l,i} \frac{(A*H^T)_{l,i}}{(W*H*H^T)_{l,i}}
		$$

In [Lee, Algortihms for Non-negative Matrix Factorisation](http://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf) ist bewiesen, dass durch die o.g. Anpassungsroutinen die Kosten $k$ monoton abnehmen und in einem Minimum konvergieren. Der Algorithmus ist jedoch nicht optimal weil das gefundene Minimum ein lokales Minimum sein kann.

 ## Vor dem Versuch zu klärende Fragen
 
 * Was versteht man unter Artikel/Wort-Matrix? Wie wird diese im aktuellen Versuch gebildet?

**Unter einer Artikel/Wort-Matrix versteht man einen zweidimensionalen Array, bei dem jede Zeile ein Dokument(Artikel) bezeichnet. Jede Spalte beschreibt dabei wie oft ein Wort bzw. eine Phrase in einem Artikel vorkommt.**  
**Diese sieht dann zum Beispiel so aus:**  
```
                                        Word1   W2   W3   W4   W5   WX
                                            +    +    +    +    +    +
                                            |    |    |    |    |    |
                                            v    v    v    v    v    v
                                          +----------------------------+
                        Article1  +--->   |  1 |  0 |  4 |  9 |  0 |  1
                                          +----------------------------+
                        Article2  +--->   |  0 |  2 |  4 |  2 |  1 |  0
                                          +----------------------------+
                        Article3  +--->   |  1 |  0 |  5 |  6 |  2 |  3
                                          +---------------------+------+
                        ArticleX  +--->   |  x |  y |  z |  v |  w |  k


```

* Wie multipliziert man die Matrix
    $$
    A= \left( \begin{array}{cccc}
a_{00} & a_{01} & a_{02} & a_{03} \\ 
a_{10} & a_{11} & a_{12} & a_{13} \\ 
a_{20} & a_{21} & a_{22} & a_{23}
\end{array} \right)
    $$
    mit dem Vektor  
    $$
    v=\left( \begin{array}{c}
v_{0} \\ 
v_{1} \\ 
v_{2} \\ 
v_{3}
\end{array} \right)
    $$
    

**Bei der Matrix-Vektor Multiplikation wird immer ein Wert einer Zeile mit einem Wert einer Spalte multipliziert und dieser Wert dann mit dem Rest der Werte aus der selben Zeile aufsummiert.**  
**Das Ergebnis hierbei ist immer ein Vektor der selben Dimension wie die des Ausgangsvektors.**

![matrixVektorMulti](./matrixvectormultiplication.png)

* Was versteht man im Kontext der NNMF unter
    * Merkmalsmatrix
    * Gewichtsmatrix

**Merkmalsmatrix:**  
Durch die Multiplikation des Merkmalsvektor, welcher wiederrum durch die Multiplikation der Matrix mit dem WordVec ensteht, mit der Matrix errechnen sich die einzelnen Merkmale die den Artikeln zugeordnet sind.

Die Merkmalsmatrix definiert also aus welchen Wörtern die Merkmale gebildet werden.  

**Gewichtsmatrix:**  
Die Gewichtsmartix beschreibt mit welchem Gewicht die einzelnen Merkmale der Merkmalsmatrix in den verschiedenen Artikeln auftreten.

* Wie definieren die Zeilen der Merkmalsmatrix die einzelnen Merkmale (Topics)?
* Was definieren die Zeilen der Gewichtungsmatrix?

**Merkmalsmatrix:**  
Jede Zeile beschreibt ein Merkmal, jede Spalte ein Wort. Je höher der Wert pro Spalte ist, desto wichtiger ist dieses Wort zur Bestimmung des jeweiligen Merkmals.

**Gewichtsmatrix:**  
Jede Zeile beschreibt einen Artikel, jede Spalte die Gewichtung des jeweiligen Merkmals. Je höher der Wert, desto präsenter ist das jeweilige Merkmal im Artikel.


* Wie werden in Numpy zwei Arrays (Typ numpy.array) 
	* im Sinne der Matrixmultiplikation miteinander multipliziert?
	* elementweise multipliziert?
* Wie wird die Transponierte eines Numpy-Arrays berechnet?

**Mit dem Befehl 'numpy.dot' lassen sich zwei Matrizen im Sinne der Matrixmultiplikation miteinander multiplizieren.**  
**Der Befehl 'numpy.multiply' multipliziert die beiden gegebenen Matrizen elementweise. Das selbe verhalten kann mit zwei numpy arrays mit 'a1 * a2' erreicht werden.**  

**Die Transponierte eines Numpy-Array lässt sich mit dem Befehl 'numpy.matrix.transpose' berechnen.**

# Versuchsdurchführung
Die in diesem Versuch einzubindenden Feeds sind in der unten stehenden Liste _feedlist_ definiert. Die aus dem vorigen Vesuch bereits bekannte Funktion _stripHTML()_ ist ebenfalls gegeben:

In [117]:
import feedparser

feedlist=['http://feeds.reuters.com/reuters/topNews',
          'http://feeds.reuters.com/reuters/businessNews',
          'http://feeds.reuters.com/reuters/worldNews',
          'http://feeds2.feedburner.com/time/world',
          'http://feeds2.feedburner.com/time/business',
          'http://feeds2.feedburner.com/time/politics',
          'http://rss.cnn.com/rss/edition.rss',
          'http://rss.cnn.com/rss/edition_world.rss',
          'http://www.nytimes.com/services/xml/rss/nyt/GlobalHome.xml',
          'http://feeds.nytimes.com/nyt/rss/Business',
          'http://www.nytimes.com/services/xml/rss/nyt/World.xml',
          'http://www.nytimes.com/services/xml/rss/nyt/Economy.xml'
          ]

In [118]:
def stripHTML(h):
  p=''
  s=0
  for c in h:
    if c=='<': s=1
    elif c=='>':
      s=0
      p+=' '
    elif s==0: p+=c
  return p

## Anlegen der Artikel/Wort-Matrix

### Die Funktion _getarticlewords()_
Schreiben Sie eine Funktion _getarticlewords()_, die folgende Elemente zurückgibt:

* _allwords:_ ist ein Dictionary dessen Keys die Worte aller gesammelten Artikel sind. Der zu jedem Key gehörende Wert ist die Anzahl, wie oft das Wort insgesamt vorkommt.
* _articlewords:_ ist eine Liste mit so vielen Elementen wie Artikel in der Sammlung sind. Jedes Listenelement ist ein Dictionary, welches die Worte des jeweiligen Artikels als Key enthält und als Wert die Worthäufigkeit.
* _articletitles_ ist eine Liste mit so vielen Elementen wie Artikel in der Sammlung sind. Jedes Element ist der Artikeltitel als String.

In [119]:
import nltk

from nltk.corpus import stopwords
stops = set(stopwords.words('english'))

In [120]:
common_words = ['said', 'today', 'yesterday', 'frequently', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday',
               'saturday', 'sunday', 'four']

In [121]:
def getwordsTFStop(docText):    
    sentence = nltk.regexp_tokenize(docText, r"[\wäöüÄÖÜß]+")
    
    tokens = [x.lower() for x in sentence if x.lower() not in stops]
    
    wordDict = { x: tokens.count(x) for x in tokens }
    
    return wordDict

Für das Parsing der Feeds soll wieder das Modul _feedparser_ eingesetzt werden. Die zu einer Nachricht gehörenden Wörter sollen die Wörter des Elements _title_ und die Wörter des Elements _description_ sein (siehe voriger Versuch). Allerdings sollen hier nicht alle Wörter eingebunden werden, sondern wie im vorigen Versuch eine Methode _getwords()_ implementiert werden, welche nur die _relevanten_ Wörter zurückgibt. Die Frage welche Wörter relevant sind ist nicht eindeutig beantwortbar. Sie können sich hierzu eigene Antworten einfallen lassen. Auf jeden Fall sollten aber die Stopwörter ignoriert werden. Hierzu kann z.B. die Stopwortliste von NLTK angewandt werden.

In [122]:
def get_article_words():
    comb_text = ''
    single_texts = []
    article_titles = []
    
    for feed in feedlist:
        f=feedparser.parse(feed)
        for e in f.entries:
            if e.get('description', 0):
                article_titles.append(e.title)
                single_texts.append(stripHTML(e.title+'\n'+e.description))
                comb_text += stripHTML(e.title+'\n'+e.description + '\n')
                
    all_words = getwordsTFStop(comb_text)
    remove_list = get_remove_list(all_words, single_texts)
    article_words = [({k: v for k, v in getwordsTFStop(text).items() if k not in remove_list}, index)\
                     for index, text in enumerate(single_texts)]
    
    #remove the title for the empty dicts
    to_remove = []
    for word_dict, index in article_words:
        if len(word_dict) == 0:
            to_remove.append(article_titles[index])
            
    article_titles = [article for article in article_titles if article not in to_remove]
            
    article_words = [w_dict for w_dict, index in article_words if len(w_dict) > 0]
        
    return (all_words, article_words, article_titles)

Nachdem alle relevanten Wörter aller Nachrichten gesammelt sind, sollte eine weitere Bereinigung stattfinden, die 

* alle Wörter, die weniger als 4 mal vorkommen
* alle Wörter, die in mehr als 30% aller Dokumente vorkommen

entfernt. 

Durch dieses Herausfiltern nicht relevanter Wörter kann es vorkommen, dass einzelne Artikel keine relevanten Wörter mehr enthalten. Diese Artikel sollen dann ganz ignoriert werden. D.h. unter anderem, dass diese Artikel auch nicht in _articlewords_ und _articletitles_ erscheinen.

In [123]:
def get_remove_list(all_words, single_texts):
    MIN_OCCURENCE = 4
    MAX_OCCURENCE = 0.3
    
    remove_list = []
    
    for key, value in all_words.items():
        key_occurrence = 0
        for text in single_texts:
            if key in text:
                key_occurrence += 1
        
        if value < 4 or key_occurrence > (len(single_texts) / 3):
            remove_list.append(key)
            del all_words[key]
    
    return remove_list

In [124]:
all_words, article_words, article_titles = get_article_words()

In [125]:
article_titles[:10]

[u"Exclusive: Risking Beijing's ire, Vietnam begins dredging on South China Sea reef",
 u'Trump expected to name CKE Restaurants CEO to head Labor Department: source',
 u'Trump to nominate Pruitt to lead U.S. environmental agency: statement',
 u'Quake magnitude 6.5 reported off coast of Northern California: USGS',
 u'U.S. jobless claims drop from five-month high',
 u'Poll: U.S. economic outlook unmoved so far after Trump win',
 u'ECB to scale back asset buys as it extends to end-2017',
 u"Syrian army's Aleppo advance slows but victory in sight",
 u'Iraqi troops retreat after Mosul hospital battle',
 u"British spy chief says Islamic State plotting attacks as Russia makes 'desert' of Syria"]

### Die Funktion _makematrix()_
Schreiben Sie eine Funktion _makematrix()_, die aus dem Dictionary _allwords_ und der Liste _articlewords_ (vorige Aufgabe) die Artikel-/Wort-Matrix generiert. Die Einträge in der Matrix sollen die Häufigkeiten der Wörter im jeweiligen Dokument sein (term frequency tf). Die Artikel-/Wort-Matrix soll als 2-dimensionales Numpy Array angelegt werden.

In [126]:
import numpy as np

In [127]:
def make_matrix(all_words, article_words):
    return np.array([[article.get(word, 0) for word in all_words.keys()] for article in article_words])

In [128]:
matrix = make_matrix(all_words, article_words)
matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## Die Nicht Negative Matrix Faktorisierung
Die Implementierung der NNMF ist entsprechend der Beschreibung im Theoriekapitel durchzuführen.

* Implementieren Sie die Funktion _cost(A,B)_. Dieser Funktion werden zwei Numpy-Matrizen $A$ und $B$ übergeben. Zurück geliefert werden die nach oben angegebener Formel berechneten Kosten $k$. Diese Funktion wird von der im folgenden beschriebenen Funktion _nnmf(A,m,it)_ benutzt.
* Implementieren Sie die Funktion __nnmf(A,m,it)__. In dieser Funktion soll der oben beschriebene Algorithmus für die Nicht-negative Matrix Faktorisierung ausgeführt werden. Der Funktion wird die zu faktorisierende Matrix $A$, die Anzahl der Merkmale $m$ und die Anzahl der Iterationen $it$ übergeben. Die Funktion gibt die gefundenen Faktoren $W$ und $H$ zurück. In jeder Iteration sollen mit der Funktion __cost(A,B)__ die Kosten berechnet werden. Sobald die Kostenabnahme pro 10 Iterationen kleiner als $2$ ist oder eine maximale Anzahl von Iterationen ($maxIt=200$) erreicht ist, soll der Algorithmus mit der Rückgabe der Faktoren $W$ und $H$ terminieren.     


Tipp für die Implementierung elementweiser Operationen von Matrizen: Für elementweise Operationen müssen in Python/Numpy nicht alle Elemente über Schleifen explizit berechnet werden. Eine elementweise Anpassung aller Matrixelemente kann kompakt programmiert werden indem die beteiligten Matrizen für diese Operationen als Arrays implementiert werden. Sollen z.B. die beiden gleich großen Numpy Arrays $U$ und $V$ elementweise multipliziert werden, dann wäre der entsprechende Programmcode einfach _U*V_.  

In [129]:
import numpy as np

matrix_A = np.random.rand(5, 5)

def nnmf_cost (A, B):
    return np.sum(np.square((A - B)))


def nnmf (A, m, maxIt):        
    matrix_W = np.random.rand(len(A), m)
    matrix_H = np.random.rand(m, len(A[0]))

    temp_A = matrix_W.dot(matrix_H)
    prev_cost = cost = nnmf_cost(A, temp_A)    
    
    for i in range(maxIt):        
        #print cost
        
        # terminate if last 10 iterations reduced costs by less than 2
        if i != 0 and i % 10 == 0:
            if (prev_cost - cost) < 2:
                return ( matrix_W, matrix_H )
            
            prev_cost = cost
        
        # recalculate H
        matrix_numerator = np.transpose(matrix_W).dot(A)
        matrix_denominator = np.transpose(matrix_W).dot(matrix_W).dot(matrix_H)
    
        for row in range(len(matrix_H)):
            for col in range(len(matrix_H[0])):                
                matrix_H[row][col] = matrix_H[row][col] * (matrix_numerator[row][col] / matrix_denominator[row][col])
        
        # recalculate W        
        matrix_numerator = A.dot(np.transpose(matrix_H))
        matrix_denominator = matrix_W.dot(matrix_H).dot(np.transpose(matrix_H))
        
        for row in range(len(matrix_W)):
            for col in range(len(matrix_W[0])):                
                matrix_W[row][col] = matrix_W[row][col] * (matrix_numerator[row][col] / matrix_denominator[row][col])
    
        ## recalculate cost
        temp_A = matrix_W.dot(matrix_H)
        cost = nnmf_cost(A, temp_A)    
    
    
    return ( matrix_W, matrix_H )

In [130]:
matrix_W, matrix_H = nnmf(matrix, 10, 200)

## Anzeige der Merkmale und der Gewichte

Im vorigen Abschnitt wurde die Merkmalsmatrix $H$ und die Gewichtsmatrix $W$ berechnet. Diese Matrizen können natürlich am Bildschirm ausgegeben werden, was jedoch nicht besonders informativ ist. Aus den Matrizen können jedoch die Antworten für die folgenden interessanten Fragen berechnet werden:

* In welchen Artikeln sind welche Merkmale stark vertreten?
* Wie lassen sich die insgesamt $m$ Merkmale beschreiben, so dass aus dieser Merkmalsbeschreibung klar wird, welches Thema den Artikeln, in denen das Merkmal stark vertreten ist, behandelt wird? 
 
Die Antwort auf die erste Frage ergibt sich aus der Gewichtsmatrix $W$. Für die Beantwortung der zweiten Frage wird die Merkmalsmatrix $H$ herangezogen.

### Beschreibung der Merkmale

Die Merkmalsmatrix $H$ beschreibt, wie stark die Worte aus _wordvec_ in jedem Merkmal enthalten sind. Jede Zeile von $H$ gehört zu einem Merkmal, jede Spalte von $H$ gehört zu einem Wort in _wordvec_.

Es bietet sich an jedes Merkmal einfach durch die $N=6$ Wörter aus _wordvec_ zu beschreiben, welche am stärksten in diesem Merkmal enthalten sind. Hierzu muss für jedes Merkmal die entsprechende Zeile in $H$ nach den $N=6$ größten Werten durchsucht bzw. geordnet werden. Die entsprechenden Spalten dieser Matrixelemente verweisen dann auf die $N=6$ wichtigsten Worte des Merkmals.

Tipp für die Implementierung: Legen Sie für jedes Merkmal $i$ eine Liste an. Die Listenlänge ist durch die Anzahl der Worte in _wordvec_ (d.h. die Anzahl der Spalten in $H$) gegeben. Jedes Listenelement $j$ enthält selbst wieder 2 Elemente: An erster Stelle den entsprechenden Wert $H_{i,j}$ der Merkmalsmatrix, an der zweiten Stelle das $j.$-te Wort in _wordvec_. Nachdem die Liste angelegt ist, kann sie mit _listname.sort()_ in aufsteigender Reihenfolge sortiert werden. Die abnehmende Sortierung erhält man mit _listname.sort().reverse()_. Danach geben die $N=6$ ersten Listenelemente die für das Merkmal $i$ wichtigsten Worte an.

   
### Präsenz der Merkmale in den Artikeln

Die Gewichtsmatrix $W$ beschreibt, wie stark die $m$ Merkmale in den Artikeln aus _articletitles_ enthalten sind. Jede Zeile von $W$ gehört zu einem Artikel, jede Spalte von $W$ gehört zu einem Merkmal.
Die Berechnung der $M=2$ gewichtigsten Merkmale für jeden Artikel in _articletitles_ kann analog zu der oben beschriebenen Berechnung der $N=6$ wichtigsten Worte eines Merkmals berechnet werden.

In [131]:
import numpy as np

np.random.seed(0)

matrix_A = np.random.rand(5, 5)

def nnmf_cost (A, B):
    return np.sum(np.square((A - B)))


def nnmf (A, m, maxIt):        
    matrix_W = np.random.rand(len(A), m)
    matrix_H = np.random.rand(m, len(A[0]))

    temp_A = matrix_W.dot(matrix_H)
    prev_cost = cost = nnmf_cost(A, temp_A)    
    
    for i in range(maxIt):      
        # terminate if last 10 iterations reduced costs by less than 
        if i != 0 and i % 10 == 0:
            if (prev_cost - cost) < 2:
                return ( matrix_W, matrix_H, cost )
            
            prev_cost = cost
            
        # recalculate H
        matrix_numerator = np.transpose(matrix_W).dot(A)
        matrix_denominator = np.transpose(matrix_W).dot(matrix_W).dot(matrix_H)
    
        for row in range(len(matrix_H)):
            for col in range(len(matrix_H[0])):                
                matrix_H[row][col] = matrix_H[row][col] * (matrix_numerator[row][col] / matrix_denominator[row][col])
        
        # recalculate W        
        matrix_numerator = A.dot(np.transpose(matrix_H))
        matrix_denominator = matrix_W.dot(matrix_H).dot(np.transpose(matrix_H))
        
        for row in range(len(matrix_W)):
            for col in range(len(matrix_W[0])):                
                matrix_W[row][col] = matrix_W[row][col] * (matrix_numerator[row][col] / matrix_denominator[row][col])
    
        ## recalculate cost
        temp_A = matrix_W.dot(matrix_H)
        cost = nnmf_cost(A, temp_A)    
    
    
    return ( matrix_W, matrix_H, cost )

In [133]:
(matrix_weights, matrix_topics, cost) = nnmf(matrix, 35, 200)


print cost

for topic in matrix_topics:
    words = all_words.keys()
    indices = np.argsort(topic)
    print [words[i] for i in indices[:6]]

1278.23130639
[u'founder', u'iraq', u'pick', u'africa', u'two', u'pearl']
[u'founder', u'small', u'cienfuegos', u'pick', u'africa', u'two']
[u'founder', u'york', u'poll', u'turkey', u'mars', u'wants']
[u'aceh', u'cienfuegos', u'africa', u'two', u'pearl', u'6']
[u'founder', u'cienfuegos', u'iraq', u'pick', u'africa', u'two']
[u'founder', u'middle', u'buys', u'trains', u'income', u'michael']
[u'founder', u'4', u'income', u'michael', u'bookstore', u'sales']
[u'aceh', u'8221', u'girl', u'oklahoma', u'cars', u'retired']
[u'founder', u'need', u'wall', u'4', u'income', u'july']
[u'aceh', u'oakland', u'airline', u'bookstore', u'buying', u'sales']
[u'founder', u'buying', u'5', u'month', u'hye', u'cienfuegos']
[u'aceh', u'nine', u'states', u'plant', u'seoul', u'plane']
[u'aceh', u'trains', u'need', u'4', u'income', u'michael']
[u'founder', u'iraq', u'two', u'pearl', u'6', u'markets']
[u'founder', u'bookstore', u'buying', u'sales', u'britain', u'5']
[u'founder', u'oakland', u'airline', u'bookstor

### Implementierung

Implementieren Sie eine Funktion _showfeatures(w,h,titles,wordvec)_, welche wie oben beschrieben für jeden Artikel die $M=2$ wichtigsten Merkmale am Bildschirm ausgibt. Dabei soll jedes Merkmal durch die 6 wichtigsten Wörter dieses Merkmals angegeben werden. Siehe Beispielausgabe unten.  

Übergabeparameter der Funktion sind die Merkmalsmatrix $H$, die Gewichtungsmatrix $W$, die Liste aller Artikeltitel _articletitles_ und die Liste aller Worte _wordvec_.


Beispiel fuer Ausgabe:

[(13.54131155883748, 13, u'Putin vows payback after confirmation of Egypt plane bomb'),

(2.2466669548146254, 9, u'Putin vows payback after confirmation of Egypt plane bomb')]

----- ['plane', 'egypt', 'russia', 'month', 'killing', 'putin']

----- ['airport', 'russian', 'crash', 'egypt', 'security', 'officials']

Die Ausgabe ist wie folgt zu interpretieren:
* Für den Artikel _Putin vows payback after confirmation of Egypt plane bomb_ ist 
    * das wichtigste Merkmal durch die 6 Wörter _plane_, _egypt_, _russia_, _month_, _killing_, _putin_ definiert. Das Gewicht dieses Merkmals im Artikel ist 13.54
    * das zweitwichtigste Merkmal durch die 6 Wörter _airport_, _russian_, _crash_, _egypt_, _security_, _officials_ definiert. Das Gewicht dieses Merkmals im Artikel ist 2.24

In [134]:
def get_for_feature(feature):
    words = all_words.keys()
    indices = (-np.array(feature)).argsort()[:6]
    return [words[i] for i in indices]

In [141]:
def show_features(w,h,titles,wordvec):
    for index, article in enumerate(article_titles[:25]): #only print 25 article-titles
        curr_w = w[index]
        
        w_indices = (-np.array(curr_w)).argsort()[:2]
        
        print article
        for i in w_indices:
            print curr_w[i], get_for_feature(h[i])
            
        print

In [142]:
show_features(matrix_W, matrix_H, article_titles, all_words)

Exclusive: Risking Beijing's ire, Vietnam begins dredging on South China Sea reef
5.83640347983 [u'south', u'president', u'korea', u'park', u'hye', u'geun']
3.25027867877 [u'state', u'islamic', u'chief', u'reuters', u'attacks', u'syria']

Trump expected to name CKE Restaurants CEO to head Labor Department: source
5.76238053155 [u'trump', u'donald', u'president', u'elect', u'j', u'win']
2.26820994406 [u'last', u'month', u'week', u'court', u'five', u'high']

Trump to nominate Pruitt to lead U.S. environmental agency: statement
14.7746612166 [u'pruitt', u'trump', u'scott', u'lead', u'environmental', u'general']
1.33355291073 [u'reuters', u'bank', u'central', u'thursday', u'2017', u'would']

Quake magnitude 6.5 reported off coast of Northern California: USGS
2.83313627527 [u'reuters', u'bank', u'central', u'thursday', u'2017', u'would']
1.99240033853 [u'state', u'islamic', u'chief', u'reuters', u'attacks', u'syria']

U.S. jobless claims drop from five-month high
9.22943931692 [u'last', u'm

## Aufgaben

1.Analysieren Sie die berechneten Topics indem Sie sich überlegen ob die gefundenen 6 Wörter pro Topic wirklich Themen beschreiben.

**Bad:**  
    - [u'aceh', u'retired', u'11', u'000', u'dredging', u'america']
    - [u'increase', u'turkey', u'mars', u'buys', u'trains', u'around']

**Bei den hier aufgeführten schlechten Topics erkennt man leider sehr wenig bis gar keinen Zusammenhang zwischen den Merkmalen. Z.b. wird 'turkey' mit 'mars' und 'trains' zusammengefasst, was leider gar keinen Sinn ergibt.**

**Good**


    - [u'pruitt', u'trump', u'scott', u'lead', u'environmental', u'general']
    - [u'state', u'islamic', u'chief', u'reuters', u'attacks', u'syria']

**Bei diesen Beispielen sind zwar auch nicht alle Begriffe thematisch zusammenhängend aber es wird klar, dass es sich bei dem ersten Beispiel um einen ein Thema mit Trum und der Klimaveränderung handelt ('Trump Picks Scott Pruitt, Climate Change Denialist, to Lead E.P.A.').**

**Beim 2. Beispiel um ein Thema in dem der Islamische Staat eine große Rolle spielt. ('British spy chief says Islamic State plotting attacks as Russia makes 'desert' of Syria')**

2.Verändern Sie die Parameter der NNMF (Anzahl der Topics $m$, Anzahl der Iterationen). Bei welcher Einstellung der Parameter erhalten Sie das für sie sinnvollste Resultat (sinnvolle Topics)?

**Man erkennt, dass die Kosten sich relativ schnell einer Asymptote annähert und man expotentiel mehr Iterationen benötigt um nur noch marginale Verbesserungen zu bekommen. 100 bis 200 Iterationen sind ausreichend, danach wird die topic extraction nicht mehr bemerkbar besser. Ein viel größerer Einfluss hat welche W-, H-Matritzen zu Beginn zufällig generiert werden.** 

```
Iterationen - Kosten

10            1273.84807468
100           1053.96711681
200           1043.08143901
400           1036.33627363
1000          1034.91014625
2000          1034.53171712
```

**Die Bewertung der Qualität der topic extraction ist natürlich auch vom persönlichen Empfinden abhängig. Wir haben    bei 30 bis 40 die besten Ergebnisse erhalten. Bei höheren Werten wie z.B. 100 nimmt die Kostenfunktion zwar ab, aber es findet auch keine relevante Gruppierung der Artikel mehr statt.**

3.Wie kann die _getwords()_ Methode verbessert werden, so dass noch bedeutsamere Topics gefunden werden? 

**Eine mögliche Optimierung wäre, dass man zusätzlich zum Entfernen der Stopwords auch noch weitere Wörter vorab Filtert.**  

**Zu diesen Worten gerhören z.B. "said"("sagte"), "today"("heute") usw. -> Also Wörter welche in Artikeln und News extrem häufig vorkommen.**  
**Außerdem sind Wochentage und allgemein Zeitangaben weniger relevant und können deshalb auch herausgefiltert werden.**  

**Natürlich sind die zusätzlich filterbaren Wörter speziell auf die gesuchten Topics anzupassen.**